In [26]:
from pathlib import Path
import pandas as pd
import glob

In [27]:
# Relative paths
current_dir = Path.cwd()
data_dir = current_dir / "data"

# Get files data
files = data_dir.glob("*.csv")

In [30]:
def get_df(data_dir:Path) -> pd.DataFrame:
    files = data_dir.glob("*csv")
    
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        dfs.append(df)
    result_df = pd.concat(dfs, ignore_index=True)
    
    return result_df 

In [29]:
df = get_df(data_dir=data_dir)